In [1]:
ENV["JULIA_PKG_USE_CLI_GIT"] = "true"
cd("/leonardo_work/Sis25_degironc_0/apol/aceconverge2025/")
pwd()

import Pkg
Pkg.activate(".")
using ACEpotentials, LinearAlgebra

  Activating project at `/leonardo_work/Sis25_degironc_0/apol/aceconverge2025`


# Basis checking

## Carbon

## Silicon 

In [5]:
dimers = read_extxyz("./datasets/Si2.xyz")

100-element Vector{Atoms}:
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 ⋮
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms

In [10]:
basis_bin = Dict()
orders = [2,3,4]
degrees = [[24,20],[24,20,16],[24,20,16,12]]
basis_tags = ["24.20","24.20.16","24.20.16.12"]

for (i,ord) in enumerate(orders)
    basis = ACE1x.ace_basis(
        elements = [:Si],
        order = ord,
        totaldegree = degrees[i],
        rcut = 6.0,
        r0 = 2.20707071, # for silicon dataset
        pure = false)
    basis_bin[basis_tags[i]] = basis
    println("basis_bin[\"" * basis_tags[i] * "\"]")
    println(length(basis))
end

basis_bin_pure = Dict()
for (i,ord) in enumerate(orders)
    basis = ACE1x.ace_basis(
        elements = [:Si],
        order = ord,
        totaldegree = degrees[i],
        rcut = 6.0,
        r0 = 2.20707071, # for silicon dataset
        pure = true)
    basis_bin_pure[basis_tags[i]] = basis
    println("basis_bin[\"" * basis_tags[i] * "\"]")
    println(length(basis))
end

basis_bin["24.20"]
297
basis_bin["24.20.16"]
802
basis_bin["24.20.16.12"]
929


[ Info: Option `pure = true` overrides `pure2b=true`


basis_bin["24.20"]
297


[ Info: Option `pure = true` overrides `pure2b=true`


basis_bin["24.20.16"]
802


[ Info: Option `pure = true` overrides `pure2b=true`


basis_bin["24.20.16.12"]
929


In [19]:
tag = "24.20.16.12"
dirty_feat = site_descriptors(basis_bin[tag], dimers[1])
pure_feat = site_descriptors(basis_bin_pure[tag], dimers[1])

2-element Vector{Vector{Float64}}:
 [0.9133427098012319, 1.1317292706625932, 0.5467092209828912, -0.38120714814071993, -1.0912737823112983, -1.1712873073419299, -0.5771123824676915, 0.3487394331699192, 1.0754790878316716, 1.1875917720470972  …  -9.107298248878237e-17, -1.9949319973733282e-17, -8.847089727481716e-17, 1.3877787807814457e-17, 3.469446951953614e-18, 2.7755575615628914e-17, 5.204170427930421e-18, -3.469446951953614e-17, -4.5102810375396984e-17, -5.551115123125783e-17]
 [0.9133427098012319, 1.1317292706625932, 0.5467092209828912, -0.38120714814071993, -1.0912737823112983, -1.1712873073419299, -0.5771123824676915, 0.3487394331699192, 1.0754790878316716, 1.1875917720470972  …  -9.107298248878237e-17, -1.9949319973733282e-17, -8.847089727481716e-17, 1.3877787807814457e-17, 3.469446951953614e-18, 2.7755575615628914e-17, 5.204170427930421e-18, -3.469446951953614e-17, -4.5102810375396984e-17, -5.551115123125783e-17]

In [1]:
println("length of dirty_feat[1]: ", length(dirty_feat[1]))
println("length of pure_feat[1]: ", length(pure_feat[1]))

LoadError: UndefVarError: `dirty_feat` not defined

# Error extraction from complete potentials

## Out-of-domain

In [48]:
dsets = ["Si2", "Si3", "Si4", "Si5"]
vset = "Si5"
ecost = "50.0"
elossweight = parse(Float64, ecost)
weights = Dict("default" => Dict("E" => elossweight, "F" => 1.0))
basisvals = ["24.20","24.20.16","24.20.16.12"]
Vref = OneBody(:Si => -7881.32677981122) # Si2 isolated energy
datakeys = (energy_key = "energy", force_key = "forces")
val_data_dir = "./datasets/$(vset).xyz"; val_data = read_extxyz(val_data_dir)
val_atoms = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in val_data]

pots = Dict()
pots_pure = Dict()
errs = Dict("train" => Dict(), "val" => Dict())
errs_pure = Dict("train" => Dict(), "val" => Dict())

for dset in dsets
    train_data_dir = "./datasets/$(dset).xyz"; train_data = read_extxyz(train_data_dir)
    train_atoms = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]
    for basisval in basisvals
        # Load potential
        pots["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)/$(basisval)/ecost$(ecost)/potential.json")
        pots_pure["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/potential.json")
        # Compute errors
        errs["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots_pure["$(dset)_$(basisval)"])
        errs["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots_pure["$(dset)_$(basisval)"])
    end
end
        



This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.523 │    0.051 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.523 │    0.051 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.330 │    0.015 │   0.000 │
├─────────┼─────────┼──────────

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.523 │    0.051 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.523 │    0.051 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.330 │    0.015 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.330 │    0.015 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬──────────┬──────────┬─────────┐
│    Type │  E [meV] │ F [eV/A] │ V [meV] │
├─────────┼──────────┼──────────┼─────────┤
│ default │ 2722.731 │    2.094 │   0.000 │
├─────────┼──────────┼──────────┼─────────┤
│     set │ 2722.731 │    2.094 │   0.000 │
└─────────┴──────────┴──────────┴─────────┘
┌─────────┬──────────┬──────────┬─────────┐
│    Type │  E [meV] │ F [eV/A] │ V [meV] │
├─────────┼──────────┼──────────┼─────────┤
│ default │ 2679.817 │    1.667 │   0.000 │
├─────────┼──────────┼──────────┼─────────┤
│     set │ 2679.817 │    1.667 │   0.000 │
└─────────┴──────────┴──────────┴─────────┘


...

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  24.424 │    0.463 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  24.424 │    0.463 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  19.875 │    0.338 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  19.875 │    0.338 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  20.280 │    0.506 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  20.280 │    0.506 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  16.300 │    0.368 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  16.300 │    0.368 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


In [51]:
basisvals

3-element Vector{String}:
 "24.20"
 "24.20.16"
 "24.20.16.12"

In [ ]:
for dset in dsets
    println("errors for $(dset):")
    for basisval in basisvals
        println("  $(dset)_$(basisval):")
        println("    Train MAE E: ", errs["train"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
        println("    Train MAE F: ", errs["train"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])
        println("    Val MAE E: ", errs["val"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
        println("    Val MAE F: ", errs["val"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])
        # println("    Train MAE E: ", errs_pure["train"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
        # println("    Train MAE F: ", errs_pure["train"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])

        # println("    Val MAE E: ", errs_pure["val"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
        # println("    Val MAE F: ", errs_pure["val"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])
    end
end


errors for Si2:
  Si2_24.20:
    Train MAE E: 0.0003295037669613521
    Train MAE F: 0.014631883962682973
    Val MAE E: 2.6798167614910544
    Val MAE F: 1.666714735904054
  Si2_24.20.16:
    Train MAE E: 0.00032950376699773185
    Train MAE F: 0.014631883959668676
    Val MAE E: 2.679816761618633
    Val MAE F: 1.6667147358113914
  Si2_24.20.16.12:
    Train MAE E: 0.00032950376699773185
    Train MAE F: 0.014631883959668676
    Val MAE E: 2.679816761876522
    Val MAE F: 1.6667147357871739
errors for Si3:
  Si3_24.20:
    Train MAE E: 7.514078691201576e-5
    Train MAE F: 0.02571718896575403
    Val MAE E: 3.804971636814831
    Val MAE F: 2.858206490306253
  Si3_24.20.16:
    Train MAE E: 8.342407291820563e-5
    Train MAE F: 0.02464874843622091
    Val MAE E: 9.989676199672992
    Val MAE F: 4.299827323724238
  Si3_24.20.16.12:
    Train MAE E: 9.917514633343671e-5
    Train MAE F: 0.02548843020945877
    Val MAE E: 19.1013800390616
    Val MAE F: 11.106770035297926
errors for Si4:

In [3]:
## Data load ## 
dset = "Si5"
ecost = "50.0"
basisvals = ["24.20","24.20.16","24.20.16.12"]
pot = load_potential("./acejulia/$(dset)/$(basisvals[3])/ecost$(ecost)/potential.json");
pot_pure = load_potential("./acejulia/$(dset)_purify/$(basisvals[3])/ecost$(ecost)/potential.json");

# Validation data
Vref = OneBody(:Si => -7881.32677981122) # Si2 isolated energy
elossweight = parse(Float64, ecost)
datakeys = (energy_key = "energy", force_key = "force")
weights = Dict("default" => Dict("E" => elossweight, "F" => 1.0))
train_data_dir = "./datasets/$(dset).xyz"; train_data = read_extxyz(train_data_dir)
test_data_dir = "./datasets/Si5.xyz"; test_data = read_extxyz(test_data_dir)
train = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]
test = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in test_data]

## Error extraction ## 
trainerrs = ACEpotentials.linear_errors(train, pot)
trainerrs_pure = ACEpotentials.linear_errors(train, pot_pure)
testerrs = ACEpotentials.linear_errors(test, pot)
testerrs_pure = ACEpotentials.linear_errors(test, pot_pure)


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  24.424 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  24.424 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  19.875 │    0.000 │   0.000 │
├─────────┼─────────┼──────────

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  20.280 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  20.280 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  16.300 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  16.300 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  24.424 │    0.000 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  24.424 │    0.000 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼

[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table
[ Info: RMSE Table
[ Info: MAE Table


Dict{String, Dict{String, Any}} with 2 entries:
  "rmse" => Dict("default"=>Dict("V"=>0.0, "E"=>0.0202798, "F"=>0.0), "set"=>Di…
  "mae"  => Dict("default"=>Dict("V"=>0.0, "E"=>0.0163002, "F"=>0.0), "set"=>Di…

## In-domain

In [8]:
function export_dimers_to_dat(dimers_dict; filename)
    for ((z1, z0), (rr, v01)) in dimers_dict
        open(filename, "w") do io
            for (r, v) in zip(rr, v01)
                println(io, "$r $v")
            end
        end
    end
end

export_dimers_to_dat (generic function with 1 method)

In [27]:
dsets = ["Si23", "Si234", "Si2345"]
vset = "Si5"
ecost = "50.0"
elossweight = parse(Float64, ecost)
weights = Dict("default" => Dict("E" => elossweight, "F" => 1.0))
basisvals = ["24.20","24.20.16","24.20.16.12"]
Vref = OneBody(:Si => -7881.32677981122) # Si2 isolated energy
datakeys = (energy_key = "energy", force_key = "forces")
val_data_dir = "./datasets/$(vset).xyz"; val_data = read_extxyz(val_data_dir)
val_atoms = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in val_data]

pots = Dict()
pots_pure = Dict()
errs = Dict("train" => Dict(), "val" => Dict())
errs_pure = Dict("train" => Dict(), "val" => Dict())

for dset in dsets
    train_data_dir = "./datasets/$(dset).xyz"; train_data = read_extxyz(train_data_dir)
    train_atoms = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]
    for basisval in basisvals
        pots["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)/$(basisval)/ecost$(ecost)/potential.json")
        pots_pure["$(dset)_$(basisval)"] = load_potential("./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/potential.json")

        # Errors
        errs["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["train"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(train_atoms, pots_pure["$(dset)_$(basisval)"])
        errs["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots["$(dset)_$(basisval)"])
        errs_pure["val"]["$(dset)_$(basisval)"] = ACEpotentials.linear_errors(val_atoms, pots_pure["$(dset)_$(basisval)"])

        # Dimers
        DSI = ACEpotentials.dimers(pots["$(dset)_$(basisval)"], [:Si,]; rr = range(0.529177, 7.0, length=200))
        D = ACEpotentials.dimers(pots_pure["$(dset)_$(basisval)"], [:Si,]; rr = range(0.529177, 7.0, length=200))
        export_dimers_to_dat(DSI, filename="./acejulia/$(dset)/$(basisval)/ecost$(ecost)/dimers.dat")
        export_dimers_to_dat(D, filename="./acejulia/$(dset)_purify/$(basisval)/ecost$(ecost)/dimercurve.dat")
    end
end




This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.


This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.4
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.7
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.605 │    0.033 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.605 │    0.033 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.319 │    0.010 │   0.000 │
├─────────┼─────────┼──────────

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.605 │    0.033 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.605 │    0.033 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.319 │    0.010 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.319 │    0.010 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │ 960.140 │    7.441 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │ 960.140 │    7.441 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │ 731.450 │    5.203 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │ 731.450 │    5.203 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


...

[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  11.641 │    0.375 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  11.641 │    0.375 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   9.026 │    0.277 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   9.026 │    0.277 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │  11.568 │    0.373 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │  11.568 │    0.373 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   8.963 │    0.276 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   8.963 │    0.276 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


In [33]:
# for dset in dsets
#     println("Train errors for $(dset):")
#     for basisval in basisvals
#         println("  $(dset)_$(basisval):")
#         # println("    Train MAE E: ", errs["train"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
#         # println("    Train MAE F: ", errs["train"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])
#         # println("    Train MAE E: ", errs_pure["train"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
#         # println("    Train MAE F: ", errs_pure["train"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])
#     end
# end
for dset in dsets
    println("Validation errors for $(dset):")
    for basisval in basisvals
        println("  $(dset)_$(basisval):")
        println("    Val MAE E: ", errs["val"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
        println("    Val MAE F: ", errs["val"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])
        # println("    Val MAE E: ", errs_pure["val"]["$(dset)_$(basisval)"]["mae"]["default"]["E"])
        # println("    Val MAE F: ", errs_pure["val"]["$(dset)_$(basisval)"]["mae"]["default"]["F"])
    end
end

Validation errors for Si23:
  Si23_24.20:
    Val MAE E: 0.7314496303867745
    Val MAE F: 5.202882369477908
  Si23_24.20.16:
    Val MAE E: 2.145457214576374
    Val MAE F: 8.785332757678953
  Si23_24.20.16.12:
    Val MAE E: 5.201990817811364
    Val MAE F: 10.867899320684742
Validation errors for Si234:
  Si234_24.20:
    Val MAE E: 0.5542576591895013
    Val MAE F: 1.1525539959336513
  Si234_24.20.16:
    Val MAE E: 0.6402825581082853
    Val MAE F: 2.9861313822092117
  Si234_24.20.16.12:
    Val MAE E: 0.5059191758755059
    Val MAE F: 3.941926090041813
Validation errors for Si2345:
  Si2345_24.20:
    Val MAE E: 0.04646340991892066
    Val MAE F: 0.4900724250143046
  Si2345_24.20.16:
    Val MAE E: 0.011582635910943282
    Val MAE F: 0.309479880769104
  Si2345_24.20.16.12:
    Val MAE E: 0.009026467484913155
    Val MAE F: 0.27748206529634856


## Plotting

# Single tests

In [ ]:
train_data_c = "./datasets/"

In [10]:
basis = ACE1x.ace_basis(
    elements = [:Si],
    order = 2,
    totaldegree = [24,20],
    rcut = 6.0,
    r0 = 2.20707071, # for silicon dataset
    pure = false)
Vref = OneBody(:Si => -7881.32677981122) 
weights = Dict("default" => Dict("E" => 50.0, "F" => 1.0))
datakeys = (energy_key = "energy", force_key = "forces")
train_data_dir = "./datasets/Si2.xyz"; train_data = read_extxyz(train_data_dir)
train = [ACEpotentials.AtomsData(t; weights=weights, v_ref=Vref, datakeys...) for t in train_data]


100-element Vector{AtomsData}:
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 atoms, "energy", "forces", nothing, Dict("E" => 50.0, "F" => 1.0), -15762.65355962244)
 AtomsData(JuLIP.Atoms with 2 ato

In [14]:
50/sqrt(2)

35.35533905932737

In [11]:
A, Y, W = ACEfit.assemble(train, basis)
W

[ Info: Assembling linear problem.
[ Info:   - Creating feature matrix with size (700, 297).
[ Info:   - Beginning assembly with processor count:  1.
Progress: 100%|█████████████████████████████████████████| Time: 0:00:32
[ Info:   - Assembly completed.
[ Info: Assembling full weight vector.


700-element Vector{Float64}:
 35.35533905932737
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0
 35.35533905932737
  1.0
  1.0
  1.0
  1.0
  1.0
  ⋮
  1.0
  1.0
  1.0
  1.0
  1.0
 35.35533905932737
  1.0
  1.0
  1.0
  1.0
  1.0
  1.0

In [12]:

solver = ACEfit.BLR()
results = ACEfit.solve(solver, W .* A, W .* Y)
pot = JuLIP.MLIPs.SumIP(Vref, JuLIP.MLIPs.combine(basis, results["C"]))

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (1.0e-8) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/8dE7C/src/types.jl:110


Iter     Function value   Gradient norm 
     0     7.826287e+02     6.780059e+02
 * time: 0.00023293495178222656
     1     7.351944e+01     1.852416e+03
 * time: 0.6013479232788086
     2    -1.340990e+02     2.513636e+03
 * time: 0.6064169406890869
     3    -1.012240e+03     2.493659e+03
 * time: 0.9000759124755859
     4    -1.014787e+03     1.057912e+03
 * time: 1.1052100658416748
     5    -1.015726e+03     8.607151e+01
 * time: 1.2043519020080566
     6    -1.015814e+03     3.208683e+02
 * time: 1.499202013015747
     7    -1.046059e+03     2.731556e+03
 * time: 1.900547981262207
     8    -1.050169e+03     5.895866e+01
 * time: 1.9061388969421387
     9    -1.050412e+03     3.115028e+02
 * time: 2.0072529315948486
    10    -1.050553e+03     1.892590e+02
 * time: 2.105297088623047
    11    -1.050578e+03     1.591120e-01
 * time: 2.2038018703460693
    12    -1.050578e+03     3.098871e-02
 * time: 2.299154043197632
    13    -1.050578e+03     3.768848e-06
 * time: 2.3070380687

JuLIP.MLIPs.SumIP{Any}(Any[OneBody{Float64}(Dict(:Si => -7881.32677981122)), PolyPairPot{Float64, ACE1.OrthPolys.TransformedPolys{Float64, ACE1.Transforms.Agnesi2Transform{Float64, Int64}, ACE1.OrthPolys.OrthPolyBasis{Float64}, PolyEnvelope{Float64}}, 1, 0}([-9.461223073737807, 4.451298402044143, 3.417304499315813, -3.9413912748613944, 1.285982099938214, -2.5381855106037348, -0.42576554262159894, 1.1689259902210356, -1.1624541618956739, 3.0747099360348202  …  3.0950383456658566, -1.907114696828274, 2.8773013663436005, -2.0816932311439382, 1.7839415423435572, -1.3336958649343704, 0.7178619926030388, -0.5315765759910964, 0.1479782356684918, -0.11070297689131842], PolyPairBasis{ACE1.OrthPolys.TransformedPolys{Float64, ACE1.Transforms.Agnesi2Transform{Float64, Int64}, ACE1.OrthPolys.OrthPolyBasis{Float64}, PolyEnvelope{Float64}}, 1}(JuLIP.Potentials.SZList{1}(AtomicNumber[<14>]), ACE1.OrthPolys.TransformedPolys{Float64, ACE1.Transforms.Agnesi2Transform{Float64, Int64}, ACE1.OrthPolys.OrthP

In [13]:
test_data_dir = "./datasets/Si5.xyz"; test_data = read_extxyz(test_data_dir)
test = [ACEpotentials.AtomsData(v; weights=weights, v_ref=Vref, datakeys...) for v in test_data];

ACEpotentials.linear_errors(train, pot)
ACEpotentials.linear_errors(test, pot)

┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.697 │    0.042 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.697 │    0.042 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.404 │    0.012 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.404 │    0.012 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


┌─────────┬──────────┬──────────┬─────────┐
│    Type │  E [meV] │ F [eV/A] │ V [meV] │
├─────────┼──────────┼──────────┼─────────┤
│ default │ 2722.917 │    2.097 │   0.000 │
├─────────┼──────────┼──────────┼─────────┤
│     set │ 2722.917 │    2.097 │   0.000 │
└─────────┴──────────┴──────────┴─────────┘
┌─────────┬──────────┬──────────┬─────────┐
│    Type │  E [meV] │ F [eV/A] │ V [meV] │
├─────────┼──────────┼──────────┼─────────┤
│ default │ 2679.989 │    1.669 │   0.000 │
├─────────┼──────────┼──────────┼─────────┤
│     set │ 2679.989 │    1.669 │   0.000 │
└─────────┴──────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


Dict{String, Dict{String, Any}} with 2 entries:
  "rmse" => Dict("default"=>Dict("V"=>0.0, "E"=>2.72292, "F"=>2.09683), "set"=>…
  "mae"  => Dict("default"=>Dict("V"=>0.0, "E"=>2.67999, "F"=>1.66915), "set"=>…

# Quick plots